In [ ]:
#Please Execute this notebook in Google Colab. I am not sure if this would work perfectly fine on local jupytr notebook as well

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
!ls
!pwd
!cd /content/drive
print('done')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
drive  sample_data
/content
done


In [ ]:
!pip install -r '/content/drive/My Drive/NLP/bert-for-tf2/requirements.txt'

In [ ]:
import sys
import random
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
import tensorflow as tf
import keras


In [ ]:
sys.path.append("/content/drive/My Drive/NLP/bert-for-tf2/")

In [ ]:
import bert
from bert.model import BertModelLayer
from bert.loader import params_from_pretrained_ckpt, load_stock_weights
from bert.tokenization.bert_tokenization import FullTokenizer

from tensorflow.keras.layers import Input, Flatten, Dense, Dropout, Lambda
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
import numpy as np
basePath = '/content/drive/My Drive/NLP/input/bert-pretrained-models/multi_cased_L-12_H-768_A-12/multi_cased_L-12_H-768_A-12/'

In [ ]:
train = pd.read_csv("/content/drive/My Drive/NLP/P1_train.csv")
test = pd.read_csv("/content/drive/My Drive/NLP/P1_test.csv")

data = train['sentence'].values
labels = train['label'].values

In [ ]:
sentenceLength = 256
totalClasses = 3
tokenPath = basePath + 'vocab.txt'
tokenizer = FullTokenizer(tokenPath, do_lower_case=False)

In [ ]:
def getTokens(data):
  temp = []
  for row in data['sentence']:
      temp.append( ["[CLS]"] + tokenizer.tokenize(str(row)) + ["[SEP]"] )
  tokens = list(map(tokenizer.convert_tokens_to_ids, temp))
  tokens = map(lambda tids: tids + [0] * (sentenceLength - len(tids)), tokens)
  tokens = [tf.convert_to_tensor(xi) for xi in list(tokens)]
  return tokens

x_train = tf.convert_to_tensor(getTokens(train))
x_test = tf.convert_to_tensor(getTokens(test))

In [ ]:
bert_params = params_from_pretrained_ckpt(basePath)
bert_layer = BertModelLayer.from_params(bert_params, name="bert")
bert_layer.apply_adapter_freeze()

def create_model(sentenceLength, classes):
    inputShape = Input(shape=(sentenceLength,), dtype='int32', name='input')
    bertLayer = bert_layer(inputShape)
    cls_out = Lambda(lambda seq: seq[:, 0, :])(bertLayer)
    dropout = Dropout(0.1)(cls_out)
    fc_1 = Dense(64, activation=tf.nn.relu)(dropout)
    dr_2 = Dropout(0.2)(fc_1)
    finalOutputShape = Dense(classes, activation='softmax')(dr_2)
    model = Model(inputShape, finalOutputShape)    
    return model

In [ ]:
model = create_model(sentenceLength, totalClasses)
model.build(input_shape=(None, sentenceLength))
load_stock_weights(bert_layer, basePath+"bert_model.ckpt")
def flatten_layers(bert_layer):
    if isinstance(bert_layer, keras.layers.Layer):
        yield bert_layer
    for layer in bert_layer._layers:
        for sub_layer in flatten_layers(layer):
            yield sub_layer

def getLayerInfo(name):
  if layer.name in ["LayerNorm", "adapter-down", "adapter-up"]:
    return True
  return False

for layer in flatten_layers(bert_layer):
  if getLayerInfo(layer.name): layer.trainable = True
  else: layer.trainable = False

bert_layer.embeddings_layer.trainable = False
model.compile(loss='sparse_categorical_crossentropy', optimizer=Adam(lr=0.00001), metrics=['accuracy'])
print(model.summary())

Done loading 196 BERT weights from: /content/drive/My Drive/NLP/input/bert-pretrained-models/multi_cased_L-12_H-768_A-12/multi_cased_L-12_H-768_A-12/bert_model.ckpt into <bert.model.BertModelLayer object at 0x7fe63e421da0> (prefix:bert). Count of weights not found in the checkpoint was: [0]. Count of weights with mismatched shape: [0]
Unused weights from checkpoint: 
	bert/embeddings/token_type_embeddings
	bert/pooler/dense/bias
	bert/pooler/dense/kernel
	cls/predictions/output_bias
	cls/predictions/transform/LayerNorm/beta
	cls/predictions/transform/LayerNorm/gamma
	cls/predictions/transform/dense/bias
	cls/predictions/transform/dense/kernel
	cls/seq_relationship/output_bias
	cls/seq_relationship/output_weights
Model: "functional_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input (InputLayer)           [(None, 256)]             0         
__________________________________________________________

In [ ]:
# checkpointName = "bert_fine-tuning.ckpt"
# cp_callback = tf.keras.callbacks.ModelCheckpoint(filepath=checkpointName, save_weights_only=True, verbose=1)

In [ ]:
model.fit(x_train, tf.convert_to_tensor(train['label']), epochs=50, batch_size=32, verbose=1)

Epoch 1/50
52/52 [==============================] - 19s 372ms/step - loss: 1.1473 - accuracy: 0.4066
Epoch 2/50
52/52 [==============================] - 19s 371ms/step - loss: 1.1125 - accuracy: 0.4313
Epoch 3/50
52/52 [==============================] - 19s 371ms/step - loss: 1.0961 - accuracy: 0.4277
Epoch 4/50
52/52 [==============================] - 19s 371ms/step - loss: 1.0786 - accuracy: 0.4410
Epoch 5/50
52/52 [==============================] - 19s 372ms/step - loss: 1.0896 - accuracy: 0.4157
Epoch 6/50
52/52 [==============================] - 19s 375ms/step - loss: 1.1155 - accuracy: 0.4000
Epoch 7/50
52/52 [==============================] - 19s 371ms/step - loss: 1.0760 - accuracy: 0.4265
Epoch 8/50
52/52 [==============================] - 19s 372ms/step - loss: 1.0851 - accuracy: 0.4229
Epoch 9/50
52/52 [==============================] - 20s 377ms/step - loss: 1.0954 - accuracy: 0.4223
Epoch 10/50
52/52 [==============================] - 19s 371ms/step - loss: 1.0964 - accura

In [ ]:
results = model.predict(x_test)
results = results.argmax(axis=-1)

In [ ]:
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score

In [ ]:
print(accuracy_score(test['label'], results))

0.465592972181552


In [ ]:
print(f1_score(test['label'], results, average = 'macro'))

0.289177875760451


In [ ]:
#Reference: https://www.kaggle.com/seriousran/twitter-sentiment-analysis-using-bert-in-tf-2/data

In [25]:
test['predicted_label'] = pd.Series(results)
test.to_csv('testing_output_proposed.csv', sep=',')